## Implementation

In [1]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

torch.manual_seed(1)

In [2]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()

input = torch.tensor(np.array(data[['Open']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close']), dtype=torch.float32)

# Example 1

## Set up a simple model manully

In [3]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-3)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[ 0.6614,  0.2669],
         [ 0.0617,  0.6213]],

        [[-0.4519, -0.1661],
         [-1.5228,  0.3817]]], requires_grad=True)
Parameter containing:
tensor([[-1.0276, -0.5631],
        [-0.8923, -0.0583]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [4]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[0.0926, 0.9074],
        [0.0485, 0.9515],
        [0.0617, 0.9383],
        [0.0953, 0.9047],
        [0.1599, 0.8401],
        [0.2337, 0.7663],
        [0.2229, 0.7771],
        [0.1927, 0.8073],
        [0.1890, 0.8110],
        [0.1924, 0.8076]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [5]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[0.1282, 0.8718],
        [0.0487, 0.9513],
        [0.0613, 0.9387],
        [0.0939, 0.9061],
        [0.1560, 0.8440],
        [0.2307, 0.7693],
        [0.2247, 0.7753],
        [0.1940, 0.8060],
        [0.1910, 0.8090],
        [0.1391, 0.8609]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [6]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[0.0148, 0.9852],
        [0.0026, 0.9974],
        [0.0043, 0.9957],
        [0.0108, 0.9892],
        [0.0340, 0.9660],
        [0.0838, 0.9162],
        [0.0767, 0.9233],
        [0.0543, 0.9457],
        [0.0521, 0.9479],
        [0.0371, 0.9629]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [7]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[0.0302, 0.1400],
         [0.1768, 0.6529]],

        [[0.0047, 0.0496],
         [0.0934, 0.8522]],

        [[0.0032, 0.0673],
         [0.0485, 0.8810]],

        [[0.0065, 0.1079],
         [0.0596, 0.8260]],

        [[0.0178, 0.1851],
         [0.0846, 0.7125]],

        [[0.0460, 0.2694],
         [0.1241, 0.5605]],

        [[0.0657, 0.2401],
         [0.1822, 0.5120]],

        [[0.0532, 0.2058],
         [0.1839, 0.5570]],

        [[0.0451, 0.2093],
         [0.1603, 0.5853]],

        [[0.0334, 0.1590],
         [0.1710, 0.6365]]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [8]:
IOHMM._baum_welch()

## See the training results

In [9]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([  768704., 10083172.], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[ 0.6614,  0.2669],
         [ 0.0617,  0.6213]],

        [[-0.4519, -0.1661],
         [-1.5228,  0.3817]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[-1.0276, -0.5631],
        [-0.8923, -0.0583]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)


## Viterbi

In [10]:
#call the viterbi algorithm
IOHMM.viterbi()